In [15]:
import requests
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import time
import random
import html5lib
import json

In [28]:
urls = [
    'https://next.rikunabi.com/kw%91%E5%8DH/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%83h%83%89%83C%83o%81%255B/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%92%CA%90M+%8DH%8E%96/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%83%255C%83t%83g%83E%83F%83A+%8AJ%94%AD/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%96%F2%8D%DC%8Et/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8E%F3%95t+%88%C4%93%E0/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8A%C5%8C%EC%8Et/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%95%DB%88%E7%8Em/?keywordsearch=1&log_f=1'    
    ]
fileNames = [
    'carpentor',
    'driver',
    'electrician',
    'software_engineer',
    'pharmacist',
    'secretary',
    'nurse',
    'childminder'
    ]

In [29]:
jobNumberFinder = 'rnn-pageNumber rnn-textXl'
jobTitleFinder = "rnn-linkText rnn-linkText--black"
jobDetailBaseUrl = "https://next.rikunabi.com"
jobDetailDescriptionFinder = 'rn3-companyOfferRecruitment__headText'
jobDescriptionAndWantedEmployeeFinder = '.rn3-companyOfferRecruitment__text'

In [30]:
# 後で削除
import socks,socket

user_agent_list = [
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]

In [31]:
response = requests.get("https://next.rikunabi.com/company/cmi1729260002/nx1_rq0021612896/?fr=cp_s00890&list_disp_no=1&leadtc=keyword_ichiran_cst_n3_ttl")
soup = BeautifulSoup(response.content, 'html5lib')
jobNumber = soup.select(jobDescriptionAndWantedEmployeeFinder)

print(jobNumber[1].get_text())

import socks,socket
from urllib.request import urlopen

socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr='127.0.0.1', port=9050)
socket.socket = socks.socksocket

html = requests.get('http://checkip.dyndns.com/').text
print(html)




                    ＼未経験・第二新卒歓迎／◆学歴不問◆経験・資格問いません◆女性活躍中！◆地域に根差して活躍したい方
                    
                    【具体的には】
                    社会人・職種・業界未経験の方歓迎

▼こんな方を歓迎します！
………………………………
◎施工管理の資格取得に挑戦したい方
◎前向きに業務に取り組める方
◎地域貢献に興味がある方

▼資格・経験が活かせます！
…………………………………
◎建築工事経験
◎建築施工管理経験
◎2級建築施工管理技士以上
                
<html><head><title>Current IP Check</title></head><body>Current IP Address: 156.146.34.193</body></html>



In [32]:
fileNameindex = 0
failedTopUrls = []
failedDetailUrls = []
errors = []
for url in urls:
    htmlNumber = 1
    data = []
    dataIndex = 1
    try:
        socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr='127.0.0.1', port=9050)
        socket.socket = socks.socksocket
        headers = {'User-Agent': random.choice(user_agent_list)}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, "html5lib")
        time.sleep(1)
        jobNumber = soup.find("span", class_=jobNumberFinder).get_text()
        print("Success - url:" + url + "jobNumber:" + jobNumber)
    except (RemoteDisconnected, ConnectionError, Exception) as e:
        print('Fail - url:' + url)
        failedTopUrls.append(url)
        errors.append(e)
        time.sleep(0.3)
        continue
    while(True):
        if(htmlNumber == 1):
            titles = soup.find_all("a", class_=jobTitleFinder)
        else:
            try:
                newUrl = url.replace('(?<=\?)(.*)(?=\/)', 'crn' + str(htmlNumber) + '.html')
                headers = {'User-Agent': random.choice(user_agent_list)}
                response = requests.get(newUrl, headers=headers, timeout=10)
                time.sleep(1)
                soup = BeautifulSoup(response.content, "html5lib")
                response.close()
                titles = soup.find_all("a", class_=jobTitleFinder)
            except (RemoteDisconnected, ConnectionError, Exception) as e:
                print('Fail - url:' + newUrl)
                errors.append(e)
                failedTopUrls.append(newUrl)
                htmlNumber = htmlNumber + 50
                if(htmlNumber - 1 >= int(jobNumber)):
                    break
                time.sleep(random.randint(2,5))
                continue
        for title in titles:
            try:
                headers = {'User-Agent': random.choice(user_agent_list)}
                jobDetailResponse = requests.get(jobDetailBaseUrl + title['href'], headers=headers, timeout=10)
                time.sleep(1)
                jobDetailSoup = BeautifulSoup(jobDetailResponse.content, "html5lib")
                jobDetailResponse.close()
                jobDescriptionSummary = None if jobDetailSoup.find("div", class_=jobDetailDescriptionFinder) is None else jobDetailSoup.find("div", class_=jobDetailDescriptionFinder).get_text()
                if jobDescriptionSummary is not None:
                    jobDescriptions = jobDetailSoup.select(jobDescriptionAndWantedEmployeeFinder)
                    jobDescriptionText = ''
                    jobWantedPersonText = ''
                    descriptionIndex = 0
                    for jobDescription in jobDescriptions:
                        if(descriptionIndex == 0):
                            jobDescriptionText = jobDescription.get_text()
                        elif(descriptionIndex == 1):
                            jobWantedPersonText = jobDescription.get_text()
                            break
                        descriptionIndex = descriptionIndex + 1                    
            except (RemoteDisconnected, ConnectionError, Exception) as e:
                print('url:' + jobDetailBaseUrl + title['href'])
                failedDetailUrls.append(newUrl)
                errors.append(e)
                continue
            if jobDescriptionSummary is None:
                try:
                    headers = {'User-Agent': random.choice(user_agent_list)}
                    jobMoreDetailResponse = requests.get(jobDetailBaseUrl + title['href'].replace('nx1', 'nx2'), headers=headers, timeout=10)
                    time.sleep(1)
                    jobDetailMoreSoup = BeautifulSoup(jobMoreDetailResponse.content, "html5lib")
                    jobMoreDetailResponse.close()                
                    jobDescriptionSummary = jobDetailMoreSoup.find("div", class_=jobDetailDescriptionFinder).get_text()
                    if jobDescriptionSummary is not None:
                        jobDescriptions = jobDetailMoreSoup.select(jobDescriptionAndWantedEmployeeFinder)
                        jobDescriptionText = ''
                        jobWantedPersonText = ''
                        descriptionIndex = 0                        
                        for jobDescription in jobDescriptions:
                            if(descriptionIndex == 0):
                                jobDescriptionText = jobDescription.get_text()
                            elif(descriptionIndex == 1):
                                jobWantedPersonText = jobDescription.get_text()
                                break
                            descriptionIndex = descriptionIndex + 1                    
                except (RemoteDisconnected, ConnectionError, Exception) as e:
                    print('url:' + jobDetailBaseUrl + title['href'].replace('nx1', 'nx2'))
                    errors.append(e)
                    continue
                
            data.append(
                {str(dataIndex) : {
                    'title': title.get_text(),
                    'description_summary':jobDescriptionSummary,
                    'description_detail':jobDescriptionText,
                    'desired_person':jobWantedPersonText
                    }})
            dataIndex = dataIndex + 1
            time.sleep(random.randint(2,5))
        htmlNumber = htmlNumber + 50
        if(htmlNumber - 1 >= int(jobNumber)):
            break
        time.sleep(random.randint(3,5))
    
    # export data into json file
    with open('./data/' + fileNames[fileNameindex] + '.json', mode='w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
    print(fileNames[fileNameindex] + ':done')
    fileNameindex = fileNameindex + 1
    
# export Top Url data into json file
with open('./data/failedTopUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedTopUrls, file, ensure_ascii=False, indent=2)
# export detailed Url data into json file
with open('./data/failedDetailUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedDetailUrls, file, ensure_ascii=False, indent=2)
# export error data into json file
with open('./data/error.json', mode='w', encoding='utf-8') as file:
    json.dump(str(errors), file, ensure_ascii=False, indent=2)


Success - url:https://next.rikunabi.com/kw%92%CA%90M+%8DH%8E%96/?keywordsearch=1&log_f=1jobNumber:1285
url:https://next.rikunabi.com/company/cmi2056085001/nx1_rq0021580431/?fr=cp_s00890&list_disp_no=38&leadtc=keyword_ichiran_cst_n5_ttl
url:https://next.rikunabi.com/company/cmi3723963001/nx1_rq0021945424/?fr=cp_s00890&list_disp_no=41&leadtc=keyword_ichiran_cst_n3_ttl
url:https://next.rikunabi.com/company/cmi2329181001/nx1_rq0021911457/?fr=cp_s00890&list_disp_no=4&leadtc=keyword_ichiran_cst_n3_ttl
url:https://next.rikunabi.com/company/cmi2056085001/nx2_rq0021580431/?fr=cp_s00890&list_disp_no=38&leadtc=keyword_ichiran_cst_n5_ttl
url:https://next.rikunabi.com/company/cmi3723963001/nx1_rq0021945424/?fr=cp_s00890&list_disp_no=41&leadtc=keyword_ichiran_cst_n3_ttl
url:https://next.rikunabi.com/company/cmi0128381001/nx1_rq0021740679/?fr=cp_s00890&list_disp_no=42&leadtc=keyword_ichiran_cst_n3_ttl
url:https://next.rikunabi.com/company/cmi0330704001/nx1_rq0021986061/?fr=cp_s00890&list_disp_no=44&l

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
myDatabase = client['jobAdDB']
myCollection = myDatabase['carpentor']
myCollection.insert_many(data)